In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import textblob, string
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import layers, models, optimizers, Sequential

In [2]:


df = pd.read_csv('L:/Thesis/Dateset/FeaturesExtraction_Compeleted.csv', encoding='latin-1')

df = df.astype(dtype={'title': 'string','url': 'string',
                                'date_published': 'string','author_name': 'string',
                                'content': 'string','label': 'string'})

cols = ['countOfPosWord', 'countOfNegWord', 'NumberOfChar','NumberOfWords','NumberOfSentences','AvgCharPerWord','AvgWordPerSent','NumberOfUpCase','NumberOfPunctuatuion','Orgtitle_contentSim','TopicModellingSim','afinn_score','vader_score_compound','FREI','FKGL','ARI','GFI','CLI']
data = pd.read_csv('L:/Thesis/Dateset/FeaturesExtraction_Compeleted.csv',  usecols = cols)

In [4]:
df['text_features']= df['title'] +' '+ df['content'] +' '+ df['author_name'] +' '+ df['date_published']
df['Complete_Data'] = df['text_features'] 

In [3]:
df['label'].value_counts()

FALSE           9576
TRUE            9559
half-true       4248
mostly-false    3437
mostly-true     3196
Name: label, dtype: Int64

## **SpellCheker**

**Data Preprocessing**

In [5]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1091)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1091)>


False

In [5]:
for row in range(len(df)):
  df['Complete_Data'][row] = re.sub(r"won\'t", "will not", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"can\'t", "can not", df['Complete_Data'][row])

    # general
  df['Complete_Data'][row] = re.sub(r"n\'t", " not", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'re", " are", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'s", " is", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'d", " would", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'ll", " will", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'t", " not", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'ve", " have", df['Complete_Data'][row])
  df['Complete_Data'][row] = re.sub(r"\'m", " am", df['Complete_Data'][row])
print("Done")

Done


In [6]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

X = df['Complete_Data']
stop_words = set(stopwords.words("english"))

import re
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()
    document= [word for word in document if word not in stop_words]
    document = [ps.stem(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)
    
df['Complete_Data'] = documents
print("Done")

Done


**Add Numeric features to text features**

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
data_scaled = scaler.fit_transform(data)

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
data_scaled = scaler.fit_transform(data)

In [9]:
numerical_data=list()
for row in range(0,len(df)):
  numerical_data.insert(row,data_scaled[row])
df['numerical_features'] = numerical_data

In [10]:
df = df.astype(dtype={'numerical_features': 'string'})

In [11]:
df['Complete_Data'] = df['Complete_Data'] + df['numerical_features']

In [12]:
df['Complete_Data'][150]

'â everi singl swing seat hous democrat endors medicareforal elect track win elect everi singl one â â unexpect elect loss hous democrat made headlin intern divis parti appeal voter â support progress polici lead member lose seat progress say â rep alexandria ocasio cortez democrat repres part bronx queen said anâ â hous democrat swing district support medicar elect without except â everi singl swing seat hous democrat endors medicareforal elect track win elect everi singl one â everi singl swing seat hous democrat endors elect track win elect everi ð ð ½ singl ð ð ½ one ð ð ½ claim gain traction twitter reader ask us look â gener swing seat competit hous district swing democrat republican control differ organ may rate differ seat competit cook polit report nonpartisan polit analysi organ highli regard commonli cite sourc final nov 2 cook list 23 swing seat held democrat incumb seek elect toss lean democrat 23 two incumb rep matt cartwright pennsylvania peter defazio oregonâ co sponsor

In [13]:
for row in range(0,len(df)):
    df['Complete_Data'][row] = re.sub(r'\[',' ',df['Complete_Data'][row])
    df['Complete_Data'][row] = re.sub(r'\]',' ',df['Complete_Data'][row])
    df['Complete_Data'][row] = re.sub(r'\n',' ',df['Complete_Data'][row])
    df['Complete_Data'][row] = re.sub(r'\s+', ' ', df['Complete_Data'][row], flags=re.I)



## **Set Max features**

In [8]:
!pip install textstat

ERROR! Session/line number was not unique in database. History logging moved to new session 3


In [14]:
import textstat
lenght=list()
for row in range(0,len(df)):
    lenght.insert(row , textstat.lexicon_count(df['Complete_Data'][row]))
df['lenght']=lenght

print('min: ', min(df['lenght']))
print('max: ', max(df['lenght']))

print('mean: {:.2f}'.format(np.mean(df['lenght'])))

print('mean + 2 *sigma: {:.2f}'.format(np.mean(df['lenght'])+ 2.0 * np.std(df['lenght'])))


max_features = int(np.mean(df['lenght'])+ 2.0 * np.std(df['lenght']))
print(max_features)

min:  37
max:  3090
mean: 459.13
mean + 2 *sigma: 965.39
965


In [15]:
encoder = preprocessing.LabelEncoder()
df['label'] = encoder.fit_transform(df['label'])

In [16]:
encoder.classes_

array(['FALSE', 'TRUE', 'half-true', 'mostly-false', 'mostly-true'],
      dtype=object)

In [17]:
y = df['label'].to_numpy()

In [18]:
train_ratio = 0.70
validation_ratio = 0.10
test_ratio = 0.20

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(df['Complete_Data'], y, test_size=0.3)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, train_size=0.2)

import tensorflow as tf
train_y = tf.keras.utils.to_categorical(y_train, num_classes=5)
valid_y = tf.keras.utils.to_categorical(y_val, num_classes=5)
test_y = tf.keras.utils.to_categorical(y_test, num_classes=5)

print(len(x_train), len(x_val), len(x_test))
print(len(train_y), len(valid_y), len(test_y))


21011 1801 7204
21011 1801 7204


## **word2vec**

In [19]:
import numpy as np


class Embeddings():
    """
    A class to read the word embedding file and to create the word embedding matrix
    """

    def __init__(self, path, vector_dimension):
        self.path = path 
        self.vector_dimension = vector_dimension
    
    @staticmethod
    def get_coefs(word, *arr): 
        return word, np.asarray(arr, dtype='float32')

    def get_embedding_index(self):
        embeddings_index = dict(self.get_coefs(*o.split(" ")) for o in open(self.path, errors='ignore'))
        return embeddings_index

    def create_embedding_matrix(self, tokenizer, vocab_size):
        """
        A method to create the embedding matrix
        """
        model_embed = self.get_embedding_index()

        embedding_matrix = np.zeros((vocab_size + 1, self.vector_dimension))
        for word, index in tokenizer.word_index.items():
            if index > max_features:
                break
            else:
                try:
                    embedding_matrix[index] = model_embed[word]
                except:
                    continue
        return embedding_matrix


In [20]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(df['Complete_Data'])
word_index = tokenizer.word_index
vocab_size = len(word_index)

In [21]:
embedding = Embeddings(
  'G:/datasets/glove.840B.300d/glove.840B.300d.txt', 
  vector_dimension = 300
)
embedding_matrix = embedding.create_embedding_matrix(tokenizer, vocab_size)

In [22]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
vocab_size = len(tokenizer.word_index) + 1
nonzero_elements / vocab_size

0.005676141204739517

In [23]:
padding_type='post'
trunc_type='post'

#train_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(train_x), maxlen=max_features, padding=padding_type, truncating=trunc_type)
#valid_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(valid_x), maxlen=max_features, padding=padding_type, truncating=trunc_type)

train_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(x_train), maxlen=max_features, padding=padding_type, truncating=trunc_type)

test_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(x_test), maxlen=max_features, padding=padding_type, truncating=trunc_type)
valid_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(x_val), maxlen=max_features, padding=padding_type, truncating=trunc_type)


In [24]:
print(len(train_seq_x), len(test_seq_x), len(valid_seq_x))


21011 7204 1801


In [ ]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('/content/drive/MyDrive/Datasets/Glove.6B/GoogleNews-vectors-negative300.bin',encoding="ISO-8859-1")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float64')
    
#num_word = 10000

# create a tokenizer 
token = text.Tokenizer(oov_token='UNK', filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True)
token.fit_on_texts(df['Complete_Data'])
word_index = token.word_index
print(len(word_index))
# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=max_features, padding=padding_type, truncating=trunc_type)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=max_features, padding=padding_type, truncating=trunc_type)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [25]:
from sklearn.utils import class_weight

def compute_classweights(target):
    """
    Computes the weights of the target values based on the samples
    :param target: Y-target variable
    :return: dictionary object
    """
    # compute class weights
    class_weights = class_weight.compute_class_weight('balanced',
                                                     np.unique(target),
                                                     target)
    
    # make the class weight list into dictionary
    weights = {}
    
    # enumerate the list
    for index, weight in enumerate(class_weights):
        weights[index] = weight
        
    return weights

# Get the class weights for the target variable
weights = compute_classweights(y)

C:\Users\Wintest\miniconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=[0 1 2 3 4], y=[0 3 0 ... 1 1 1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [26]:
weights

{0: 0.6269005847953216,
 1: 0.628015482791087,
 2: 1.4131826741996234,
 3: 1.7466395112016293,
 4: 1.8783479349186483}

In [27]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)
    

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [29]:
import tensorflow
def scheduler(epoch, lr):
    
    if epoch < 7:
        print('-- lr =%f --',lr) 
        return lr

    elif (epoch%7 == 0):
        lr = lr * tensorflow.math.exp(-0.2)
        print('** lr =%f **',lr)  
        return lr
        
    else:
        print('__ lr =%f __',lr) 
        return lr
    
    
callback = tensorflow.keras.callbacks.LearningRateScheduler(scheduler)

In [30]:
import tensorflow as tf
embedding_dim =300
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=max_features, weights=[embedding_matrix], trainable=False))
model.add(layers.BatchNormalization())
model.add(layers.Bidirectional(layers.LSTM(128,return_sequences=True)))
model.add(layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L1L2(l2=0.002)))
model.add(layers.GlobalMaxPool1D()) # Remove flatten layer
model.add(layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L1L2(l2=0.002)))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.L1L2(l2=0.002)))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(5, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 965, 300)          48941700  
_________________________________________________________________
batch_normalization (BatchNo (None, 965, 300)          1200      
_________________________________________________________________
bidirectional (Bidirectional (None, 965, 256)          439296    
_________________________________________________________________
dense (Dense)                (None, 965, 128)          32896     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0

In [31]:
from tensorflow.keras import losses

model.compile(optimizer=tf.keras.optimizers.Adam(8e-6), 
                  loss=losses.CategoricalCrossentropy(), 
                  metrics=[tf.metrics.AUC(name='accuracy')])

In [34]:
history = model.fit(train_seq_x, 
                    train_y,
                    batch_size=4,
                    epochs=50,
                    verbose=1,
                    validation_data=(valid_seq_x, valid_y),
                    class_weight=weights,
                    callbacks=[callback])
loss, accuracy = model.evaluate(train_seq_x, train_y, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(valid_seq_x, valid_y, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 21011 samples, validate on 1801 samples
-- lr =%f -- 7.999999979801942e-06
Epoch 1/50
21011/21011 [==============================] - 619s 29ms/sample - loss: 2.1514 - accuracy: 0.5989 - val_loss: 1.9254 - val_accuracy: 0.7854
-- lr =%f -- 7.999999979801942e-06
Epoch 2/50
21011/21011 [==============================] - 618s 29ms/sample - loss: 1.8945 - accuracy: 0.7546 - val_loss: 1.6967 - val_accuracy: 0.8606
-- lr =%f -- 7.999999979801942e-06
Epoch 3/50
21011/21011 [==============================] - 616s 29ms/sample - loss: 1.6914 - accuracy: 0.8229 - val_loss: 1.5082 - val_accuracy: 0.8885
-- lr =%f -- 7.999999979801942e-06
Epoch 4/50
21011/21011 [==============================] - 617s 29ms/sample - loss: 1.5576 - accuracy: 0.8533 - val_loss: 1.3465 - val_accuracy: 0.9034
-- lr =%f -- 7.999999979801942e-06
Epoch 5/50
21011/21011 [==============================] - 616s 29ms/sample - loss: 1.4120 - accuracy: 0.8811 - val_loss: 1

** lr =%f ** tf.Tensor(2.409554e-06, shape=(), dtype=float32)
Epoch 43/50
21011/21011 [==============================] - 615s 29ms/sample - loss: 0.6487 - accuracy: 0.9875 - val_loss: 0.7087 - val_accuracy: 0.9824
__ lr =%f __ 2.409554099358502e-06
Epoch 44/50
21011/21011 [==============================] - 617s 29ms/sample - loss: 0.6497 - accuracy: 0.9873 - val_loss: 0.7078 - val_accuracy: 0.9823
__ lr =%f __ 2.409554099358502e-06
Epoch 45/50
21011/21011 [==============================] - 615s 29ms/sample - loss: 0.6438 - accuracy: 0.9877 - val_loss: 0.7074 - val_accuracy: 0.9824
__ lr =%f __ 2.409554099358502e-06
Epoch 46/50
21011/21011 [==============================] - 616s 29ms/sample - loss: 0.6376 - accuracy: 0.9882 - val_loss: 0.7051 - val_accuracy: 0.9820
__ lr =%f __ 2.409554099358502e-06
Epoch 47/50
21011/21011 [==============================] - 615s 29ms/sample - loss: 0.6312 - accuracy: 0.9885 - val_loss: 0.7079 - val_accuracy: 0.9816
__ lr =%f __ 2.409554099358502e-06
Epo

In [35]:
model.evaluate(test_seq_x,test_y, batch_size=4)

7204/7204 [==============================] - 86s 12ms/sample - loss: 0.6877 - accuracy: 0.9822


[0.6876923868154433, 0.9821647]

In [36]:
pred_LSTM =np.argmax(model.predict(test_seq_x), axis=-1)
pred_LSTM


array([2, 0, 4, ..., 0, 1, 1], dtype=int64)

In [37]:
y_test

array([2, 0, 4, ..., 0, 1, 1])

In [40]:
y_eval = y_test
y_eval

array([2, 0, 4, ..., 0, 1, 1])

In [41]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

matrix = classification_report(y_eval,pred_LSTM)
print(matrix)

              precision    recall  f1-score   support

           0       0.85      0.90      0.88      2259
           1       0.97      0.86      0.91      2341
           2       0.73      0.84      0.78      1004
           3       0.83      0.78      0.81       825
           4       0.85      0.88      0.86       775

    accuracy                           0.86      7204
   macro avg       0.85      0.85      0.85      7204
weighted avg       0.87      0.86      0.86      7204



In [42]:
confusionmatrix = confusion_matrix(y_eval,pred_LSTM)
print(confusionmatrix)

[[2044   39   99   66   11]
 [ 172 2003  110   18   38]
 [ 106   12  842   27   17]
 [  65    2   57  646   55]
 [  16    9   47   21  682]]
